# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file = "../WeatherPy/city_list.csv"
data_file_df = pd.read_csv(data_file)
data_file_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Wad Medani,71,SD,1597704601,71,14.40,33.52,81.21,10.78
1,1,Mataura,95,NZ,1597704601,97,-46.19,168.86,35.01,4.00
2,2,Constitución,6,CL,1597704601,80,-35.33,-72.42,48.96,11.59
3,3,Nikolskoye,1,RU,1597704602,93,59.70,30.79,53.60,4.47
4,4,New Norfolk,79,AU,1597704602,91,-42.78,147.06,48.00,2.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = data_file_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = data_file_df["Humidity"]

In [6]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 300, point_radius =5)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Cities will be reduced based on the following weather condictions
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

reduced_cities_df = data_file_df.loc[(data_file_df["Wind Speed"] < 10) & (data_file_df["Cloudiness"] == 0) & \
                                   (data_file_df["Max Temp"] > 70) & (data_file_df["Max Temp"] < 80)].dropna()

reduced_cities_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
30,30,Hervey Bay,0,AU,1597704606,62,-25.30,152.85,71.01,4.70
215,215,Kumul,0,CN,1597704631,21,42.80,93.45,73.49,8.84
231,231,Ajdabiya,0,LY,1597704633,61,30.76,20.23,76.71,9.31
240,240,Tiznit Province,0,MA,1597704635,36,29.58,-9.50,71.22,1.59
372,372,Paratinga,0,BR,1597704653,42,-12.69,-43.18,79.50,8.46
453,453,Kiryat Gat,0,IL,1597704664,47,31.61,34.76,75.99,1.68


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = reduced_cities_df.loc[:,["City", "Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
30,Hervey Bay,AU,-25.30,152.85,
215,Kumul,CN,42.80,93.45,
231,Ajdabiya,LY,30.76,20.23,
240,Tiznit Province,MA,29.58,-9.50,
372,Paratinga,BR,-12.69,-43.18,
453,Kiryat Gat,IL,31.61,34.76,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [10]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No hotel")
        
    print("------------")


Retrieving Results for Index 30: Hervey Bay.
Closest hotel in Hervey Bay is Oaks Hervey Bay Resort and Spa.
------------
Retrieving Results for Index 215: Kumul.
Closest hotel in Kumul is 哈密环球大酒店.
------------
Retrieving Results for Index 231: Ajdabiya.
Closest hotel in Ajdabiya is Almadina Hotel.
------------
Retrieving Results for Index 240: Tiznit Province.
No hotel
------------
Retrieving Results for Index 372: Paratinga.
Closest hotel in Paratinga is Pousada Magalhães.
------------
Retrieving Results for Index 453: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
------------


In [11]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
30,Hervey Bay,AU,-25.30,152.85,Oaks Hervey Bay Resort and Spa
215,Kumul,CN,42.80,93.45,哈密环球大酒店
231,Ajdabiya,LY,30.76,20.23,Almadina Hotel
240,Tiznit Province,MA,29.58,-9.50,
372,Paratinga,BR,-12.69,-43.18,Pousada Magalhães
453,Kiryat Gat,IL,31.61,34.76,Desert Gat


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))